In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

In [ ]:
#### CALL BERDASARKAN LAG ####

In [2]:
# Load the training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Training Data Call")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Testing Data Call")

In [3]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}


# Define lag configurations
lags = {
    'lag1': ['RVlag1', 'BVlag1'],
    'lag7': ['RVlag7', 'BVlag7', 'SJVlag7'],
    'lag15': ['RVlag15', 'BVlag15', 'SJVlag15'],
    'lag30': ['RVlag30', 'BVlag30', 'SJVlag30']
}

In [4]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCCall_Lags.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [6]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Asset Price', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data[selected_features].values
    y_train = train_data['Last'].values
    x_test = test_data[selected_features].values
    y_test = test_data['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: lag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 5
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77736	test's gamma: 9.7975
Lag: lag1 | RMSE: 295.6539 | MAPE: 2.10%

Testing with lag: lag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1339
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2498]	train's gamma: 9.77746	test's gamma: 9.7977
Lag: lag7 | RMSE: 369.4301 | MAPE: 2.52%

Testing with lag: lag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000134 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1339
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77746	test's gamma: 9.79774
Lag: lag15 | RMSE: 343.4188 | MAPE: 2.51%

Testing with lag: lag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000054 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1336
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77747	test's gamma: 9.79767
Lag: lag30 | RMSE: 373.5035 | MAPE: 2.47%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCCall_Lags.xlsx


In [ ]:
#### CALL BERDASARKAN JENIS VARIATION ####

In [7]:
# Load training and testing data
# Load the training and testing data
train_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Training Data Call")
test_data = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Call.xlsx", sheet_name="Testing Data Call")

# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

In [8]:
# Define the variations and their respective lag features
variations = {
    'SJV': ['SJVlag1', 'SJVlag7', 'SJVlag15', 'SJVlag30'],
    'BV': ['BVlag1', 'BVlag7', 'BVlag15', 'BVlag30'],
    'RV': ['RVlag1', 'RVlag7', 'RVlag15', 'RVlag30']
}

In [9]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCCall_Variations.xlsx"

In [10]:
# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Variation", "RMSE", "MAPE"])  # Add headers for summary

In [11]:
# Loop through each variation
for var_name, features in variations.items():
    print(f"\nTesting with variation: {var_name}")
    
    # Select relevant features
    selected_features = ['Asset Price', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data[selected_features].values
    y_train = train_data['Last'].values
    x_test = test_data[selected_features].values
    y_test = test_data['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Variation: {var_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([var_name, rmse, mape])
    
    # Add sheet for this variation
    ws = wb.create_sheet(title=var_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with variation: SJV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000597 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1591
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77734	test's gamma: 9.79747
Variation: SJV | RMSE: 286.8801 | MAPE: 2.03%

Testing with variation: BV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1595
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77735	test's gamma: 9.79751
Variation: BV | RMSE: 280.0938 | MAPE: 2.08%

Testing with variation: RV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1593
[LightGBM] [Info] Number of data points in the train set: 8228, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.399805
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.77733	test's gamma: 9.79748
Variation: RV | RMSE: 297.6765 | MAPE: 2.01%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCCall_Variations.xlsx


In [ ]:
#### PUT BERDASARKAN LAG ####

In [12]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Testing Data Put")

In [13]:
# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

# Define lag configurations
lags = {
    'lag1': ['RVlag1', 'BVlag1'],
    'lag7': ['RVlag7', 'BVlag7', 'SJVlag7'],
    'lag15': ['RVlag15', 'BVlag15', 'SJVlag15'],
    'lag30': ['RVlag30', 'BVlag30', 'SJVlag30']
}

In [14]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCPut_Lags.xlsx"

# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Lag", "RMSE", "MAPE"])  # Add headers for summary

In [15]:
# Loop through each lag configuration
for lag_name, features in lags.items():
    print(f"\nTesting with lag: {lag_name}")
    
    # Select relevant features
    selected_features = ['Asset Price', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data_put[selected_features].values
    y_train = train_data_put['Last'].values
    x_test = test_data_put[selected_features].values
    y_test = test_data_put['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Lag: {lag_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data_put['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([lag_name, rmse, mape])
    
    # Add sheet for this lag
    ws = wb.create_sheet(title=lag_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with lag: lag1
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000586 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1087
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 5
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08738	test's gamma: 8.97395
Lag: lag1 | RMSE: 382.2567 | MAPE: 3.99%

Testing with lag: lag7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000528 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1342
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08767	test's gamma: 8.97454
Lag: lag7 | RMSE: 549.3515 | MAPE: 4.56%

Testing with lag: lag15
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000305 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1341
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08768	test's gamma: 8.97484
Lag: lag15 | RMSE: 480.8143 | MAPE: 4.52%

Testing with lag: lag30
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000288 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1337
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 6
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08757	test's gamma: 8.97431
Lag: lag30 | RMSE: 472.3001 | MAPE: 4.32%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCPut_Lags.xlsx


In [ ]:
#### PUT BERDASARKAN JENIS VARIATION ####

In [16]:
# Read training and testing data for Put options
train_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Training Data Put")
test_data_put = pd.read_excel("/Users/shannenwibisono/Desktop/-SKRIPSI-/Bitcoin/Bitcoin Training Testing Put.xlsx", sheet_name="Testing Data Put")

# Define the LightGBM parameters
params = {
    'boost_from_average': True,
    'colsample_bytree': 0.5,
    'fair_c': 1,
    'huber_delta': 1,
    'learning_rate': 0.05,
    'max_bin': 255,
    'max_delta_step': 0.7,
    'max_depth': -1,  # None is represented by -1 in LightGBM
    'min_child_samples': 10,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'n_estimators': 2500,
    'num_leaves': 16,
    'objective': 'gamma',
    'reg_alpha': 0,
    'reg_lambda': 0,
    'subsample': 1,
    'subsample_for_bin': 50000,
    'subsample_freq': 1,
    'tweedie_variance_power': 1.5,
    'early_stopping_round': 200  # Set early stopping in parameters
}

In [17]:
# Define the variations and their respective lag features
variations = {
    'SJV': ['SJVlag1', 'SJVlag7', 'SJVlag15', 'SJVlag30'],
    'BV': ['BVlag1', 'BVlag7', 'BVlag15', 'BVlag30'],
    'RV': ['RVlag1', 'RVlag7', 'RVlag15', 'RVlag30']
}

In [18]:
# Output file path
output_file = "/Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCPut_Variations.xlsx"

In [19]:
# Create a workbook to save results
wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"
ws_summary.append(["Variation", "RMSE", "MAPE"])  # Add headers for summary

In [20]:
# Loop through each variation
for var_name, features in variations.items():
    print(f"\nTesting with variation: {var_name}")
    
    # Select relevant features
    selected_features = ['Asset Price', 'Maturity', 'Strike'] + features
    
    # Prepare features and target
    x_train = train_data_put[selected_features].values
    y_train = train_data_put['Last'].values
    x_test = test_data_put[selected_features].values
    y_test = test_data_put['Last'].values
    
    # Create LightGBM datasets
    train_set = lgb.Dataset(x_train, label=y_train)
    test_set = lgb.Dataset(x_test, label=y_test, reference=train_set)
    
    # Train the model
    model = lgb.train(
        params, 
        train_set, 
        valid_sets=[train_set, test_set], 
        valid_names=["train", "test"], 
        num_boost_round=2500
    )
    
    # Make predictions
    y_pred = model.predict(x_test, num_iteration=model.best_iteration)
    
    # Calculate RMSE and MAPE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f"Variation: {var_name} | RMSE: {rmse:.4f} | MAPE: {mape:.2f}%")
    
    # Save results in a new sheet
    results = pd.DataFrame({
        'Date': test_data_put['Time'],  # Adjust 'Time' to your date column name
        'Actual': y_test,
        'Predicted': y_pred,
        'RMSE': [rmse] * len(y_test),  # Add RMSE for consistency
        'MAPE': [mape] * len(y_test)  # Add MAPE for consistency
    })
    
    # Add summary results
    ws_summary.append([var_name, rmse, mape])
    
    # Add sheet for this variation
    ws = wb.create_sheet(title=var_name)
    for r in dataframe_to_rows(results, index=False, header=True):
        ws.append(r)

# Save the workbook
wb.save(output_file)
print(f"All results saved to Excel file: {output_file}")


Testing with variation: SJV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000363 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1592
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08728	test's gamma: 8.97375
Variation: SJV | RMSE: 311.0357 | MAPE: 3.64%

Testing with variation: BV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000245 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1597
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08724	test's gamma: 8.97366
Variation: BV | RMSE: 360.0668 | MAPE: 3.67%

Testing with variation: RV
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000307 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1596
[LightGBM] [Info] Number of data points in the train set: 8446, number of used features: 7
[LightGBM] [Warning] Unknown parameter: huber_delta
[LightGBM] [Info] Start training from score 9.242772
Training until validation scores don't improve for 200 rounds


/Users/shannenwibisono/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Did not meet early stopping. Best iteration is:
[2500]	train's gamma: 9.08722	test's gamma: 8.97372
Variation: RV | RMSE: 351.9607 | MAPE: 3.63%
All results saved to Excel file: /Users/shannenwibisono/Desktop/-SKRIPSI-/Results LGBM/Results_LightGBM_BTCPut_Variations.xlsx
